# Data for TRAPPIST-1 system
Source: Nature 2017

In [75]:
import pandas as pd
import yaml
import numpy as np

In [76]:
with open('bodies.yaml', 'r') as f:
    raw_planet_data = yaml.load(f)

In [77]:
earth = {'mass':5.972e24,
         'radius': 6.371e6
        }
sun = {'mass':1.989e30,
       'radius': 695.8e6
      }

conversions = {'meter2au': 6.68459e-12}

In [78]:
columns = ['name', 'mass', 'period', 'radius', 'semi-major', 'eccentricity']

planet_data = pd.DataFrame(columns=columns)
star_data = pd.DataFrame(columns=columns)

for d in raw_planet_data.keys():
    current_dict = raw_planet_data[d]
    current_dict['name'] = d
    for c in current_dict.keys():
        if current_dict[c] == 'NA':
            current_dict[c] = np.nan
    if d == 'star':
        pop_columns = [current_dict['name']] + \
            [float(x) for x in [current_dict['mass'], np.nan, 
                                current_dict['radius'],np.nan,np.nan]]
        df = pd.DataFrame([pop_columns], columns=columns)
        star_data = star_data.append(df)
        continue
    pop_columns = [current_dict['name']] + \
            [float(x) for x in [current_dict['mass'], current_dict['period'], 
                                 current_dict['radius'], current_dict['semi-major'], 
                                 current_dict['eccentricity']]]        
    df = pd.DataFrame([pop_columns], columns=columns)
    planet_data = planet_data.append(df, ignore_index=True)

## Convert to local units

Using TRAPPIST units:
- time: earth days
- distance 1/1000 AU
- mass: TRAPPIST-1 mass

In [79]:
star_data['mass'][0]

0.080199999999999994

In [80]:
mass_star = star_data['mass'][0]  # in solar masses
system = planet_data.copy()
system['mass'] *= earth['mass']/sun['mass']
system = system.append(star_data[star_data['name'] == 'star'], ignore_index=True)
system['mass'] /= mass_star

In [81]:
system

,name,mass,period,radius,semi-major,eccentricity
0,d,0.000015,4.049610,0.772,21.44,0.070
1,f,0.000025,9.206690,1.045,37.10,0.063
2,c,0.000052,2.421823,1.056,15.21,0.083
3,g,0.000050,12.352940,1.127,45.10,0.061
4,b,0.000032,1.510871,1.086,11.11,0.081
5,e,0.000023,6.099615,0.918,28.17,0.085
6,h,NaN,20.000000,0.755,63.00,NaN
7,star,1.000000,NaN,0.117,NaN,NaN


In [94]:
system.to_hdf("trappist_bodies.hdf5", "LOCAL_UNITS")

## Convert to solar system units:

In [86]:
planet_data['mass'] *= earth['mass']/sun['mass'] # in M_s
planet_data['radius'] *= earth['radius'] * conversions['meter2au'] # in AU
planet_data['semi-major'] *= 10e-3 # in AU
planet_data['period'] /= 365.242 # in solar years

star_data['radius'] *= sun['radius'] * conversions['meter2au'] # in AU

In [87]:
bodies = planet_data.append(star_data[star_data['name'] == 'star'], ignore_index=True)
bodies.sort_values('name', inplace=True)
bodies

,name,mass,period,radius,semi-major,eccentricity
4,b,7.662826e-12,0.000011,1.969675e-09,0.001111,0.081
2,c,1.244082e-11,0.000018,1.915264e-09,0.001521,0.083
0,d,3.696187e-12,0.000030,1.400174e-09,0.002144,0.070
5,e,5.589355e-12,0.000046,1.664974e-09,0.002817,0.085
1,f,6.130261e-12,0.000069,1.895313e-09,0.003710,0.063
3,g,1.208022e-11,0.000093,2.044037e-09,0.004510,0.061
6,h,NaN,0.000150,1.369341e-09,0.006300,NaN
7,star,8.020000e-02,NaN,2.531071e-06,NaN,NaN


In [92]:
bodies.to_hdf("trappist_bodies.hdf5", "SOLAR_UNITS")